In [2]:
import sys
!{sys.executable} -m pip install openai

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for openai: filename=openai-0.26.4-py3-none-any.whl size=67767 sha256=d6f3bbe2c393946cb45be24e0adde234060db7e932a28246b5bfc4c18b84e264
  Stored in directory: c:\users\datapiper\appdata\local\pip\cache\wheels\6c\15\7e\10950b76450328e6997e4319968c5412a28569ccf33b1e4a04
Successfully built openai


In [3]:
import os
import openai
openai.api_key = "Enter Your OPENAI APIKEY generated from https://platform.openai.com/account/api-keys"

In [4]:
#OPENAI APIs
def openai_completion(prompt):
    """
    OpenAI text completion API given prompt return text
    """
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      max_tokens=150,
      temperature=0.5
    )
    return response['choices'][0]['text']

def openai_image(prompt):
    """
    OpenAI image generation API (DELL-E 2) given prompt return image URL
    """
    response = openai.Image.create(
      prompt=prompt,
      n=1,
      size="512x512"
    )
    image_url = response['data'][0]['url']
    return image_url

In [6]:
!{sys.executable} -m pip install panel

# What is Panel? Panel is the dashboarding library in the open-source HoloViz ecosystem that offers high-level tools to simplify visualization in Python and was developed by Philipp Rudiger, Jim Bednar, and a community of contributors. Check out panel.holoviz.org to learn more.

In [7]:
# PANEL DASHBOARD
import panel as pn
pn.extension(loading_spinner='dots', loading_color='#00aa41')

inp = pn.widgets.TextInput(value="", placeholder='Enter text here...')
button_conversation = pn.widgets.Button(name="Chat!")
button_image = pn.widgets.Button(name="Image!")
convos_text = [] # store all texts in a list
convos = [] # store all panel objects in a list

def get_conversations(_):
    prompt = inp.value
    inp.value = ''
    if prompt != "":
        convos_text.append(prompt)
        openai_answer = openai_completion('\n'.join(convos_text)) # prompt includes all history
        convos_text.append(openai_answer)
        convos.append(
            pn.Row('\U0001F60A', pn.pane.Markdown(prompt, width=600))
        )
        convos.append(
            pn.Row('\U0001F916', pn.pane.Markdown(openai_answer, width=600, style={'background-color': '#F6F6F6'}))
        )
    if len(convos_text) == 0:
        convos.append(pn.Row('\U0001F916', pn.pane.Markdown("Give me something!", width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*convos)


def get_image(_):
    if len(convos_text)>0:
        image_prompt = convos_text[-1]
        image_url = openai_image(image_prompt)
        return pn.pane.PNG(image_url, width=600)

In [8]:
interactive_conversation = pn.bind(get_conversations, button_conversation)
interactive_image = pn.bind(get_image, button_image)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation,button_image),
       pn.panel(interactive_conversation, loading_indicator=True, height=500),
    pn.panel(interactive_image, loading_indicator=True, height=500),
    
)

dashboard.servable()

Column
    [0] TextInput(placeholder='Enter text here...')
    [1] Row
        [0] Button(name='Chat!')
        [1] Button(name='Image!')
    [2] ParamFunction(function, height=500, loading_indicator=True)
    [3] ParamFunction(function, height=500, loading_indicator=True)